In [ ]:
import boto3 
import numpy as np
import pandas as pd
from skimage.color import rgb2lab, lab2rgb
import os
from io import BytesIO
from tqdm import tqdm
from PIL import Image
import pickle
from sklearn.cluster import KMeans
import itertools

In [ ]:
s3 = boto3.client('s3')

In [ ]:
def get_s3_keys_as_generator(bucket):
    """Generate all the keys in an S3 bucket."""
    kwargs = {'Bucket': bucket}
    while True:
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            yield obj['Key']

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [ ]:
bucket_name = 'model-core-data'
all_keys = list(get_s3_keys_as_generator(bucket_name))

In [ ]:
palette_dict_keys = [
    key for key in all_keys if key.startswith('palette_similarity/palette_dict_')
]

# get all the dicts

In [ ]:
a = s3.get_object(
        Bucket='model-core-data', 
        Key=palette_dict_keys[3]
    )['Body'].read()


In [ ]:
palette_dict = {}

for key in tqdm(palette_dict_keys):
    chunk_bytes = s3.get_object(
        Bucket='model-core-data', 
        Key=key
    )['Body'].read()
    chunk = pickle.loads(chunk_bytes)
    palette_dict.update(chunk)

In [ ]:
len(palette_dict)

In [ ]:
palettes = np.stack(palette_dict.values())

In [ ]:
image_ids = list(palette_dict.keys())

In [ ]:
len(palette_dict)

# save them

In [ ]:
np.save('/storage/data/palette/all_palettes', palettes)
np.save('/storage/data/palette/all_image_ids', np.array(image_ids))